# Chapter11:統計的検定
母集団の母数に対し仮定を立てて, その仮説を標本から検証する手法.

例として,   
内容量平均130gと公表されているフライドポテトが122.02gしか入っていなく
14個の買ってきて平均を計算すると128.451となった, これは偶然なのかどうか？

これを仮説検定を用いる.

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
%matplotlib inline

np.random.seed(1111)

In [3]:
df = pd.read_csv('../data/ch11_potato.csv')
sample = np.array(df['重さ'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [4]:
s_mean = np.mean(sample)
s_mean

128.4507142857143

## 11.1 統計的仮説検定とは
母集団の母数に関する二つの仮説を立て標本から計算される統計量からどちらの仮説が正しいか判断する統計学的な枠組みである. 

### 11.1.1:統計的仮説検定の基本
確かめたいことは母平均が130gより少ないかどうかの所.  
前提として母分散は9と分かっているものとする.  

ここでは  
「母平均は130g」という仮説を立てる14個のの標本は  
$$
X_1,X_2,...,X_{14} \sim^{iid}N(130,9)
$$
に従い標本平均$\overline{X}\sim N(130, 9/14)$に従う事になる.  
ここで$\overline{X}$が$P(\overline{X} \leq x)=0.05$を満たすxを考える.

In [6]:
rv=stats.norm(130, np.sqrt(9/14))
print(rv.isf(0.95))
print(rv.isf(0.995))

128.68118313069039
127.93474198413732


$\overline{X}$が$P(\overline{X} \leq 128.681)=0.05$という結果になり
123.681以下になる確率は5%の確率でしか発生しないので例でいうところの14個の標本平均が128.451になることは相当稀である.
このことから「母平均が130gより少ない」という結論をづける.というのが仮説検定の流れである.   

仮説検定では母数に関する2つの仮説, **帰無仮説**と**対立仮説**を使う.   

- 対立仮説$H_0$：主張したい仮説「差がある」, 「効果がある」
- 帰無仮説$H_1$：主張したい仮説と逆「差がない」, 「効果がない」

このどちらかを検証するために標本からの統計量を計算して仮説検定を行う.  
その結果得られるのが以下の二つ

- 帰無仮説を棄却する：帰無仮説が正しくないという**結論**
- 帰無仮説を採択する：帰無仮説が**正しいとは言えない**という**保留**の結論

それらの判断は以上の例でも検証した様に標本からの統計量が珍しいかどうかできまる.  

- 珍しい値が得られた：帰無仮説を棄却する → **有意である**
- 珍しい値が得られない：帰無仮説を採択する

これを最初の例に当てはめてみると  
- 対立仮説$H_0$：「母平均は130gより少ない」
- 帰無仮説$H_1$：「母平均は130g」

結論は
- 帰無仮説を棄却する：「母平均は130gより少ない」という**結論**
- 帰無仮説を採択する：「母平均は130g」が**正しいとは言えない**という**保留**の結論

128.451gはとなるのは有意である.  
つまり128.681より標本平均が小さくなるのは有意で**棄却域**と言われる範囲である.  
逆に128.681以上は**採択域**と呼ばれる.  

仮説検定では棄却域に入る確率を決めてから行う. この確率を**有意水準**といいこの閾値のことを**臨海値**という.  
また検定に使われる検定量を**検定統計量**というという. 

最初の例に当てはめると
有意水準は5%, 臨海値が128.681となっていた.  
検定統計量の左側の領域にも名前がついており**p値**という.  

標本平均$\overline{X}$は$N(130, 9/14)$に従うので, 検定統計量に標本平均$\overline{X}$を用いたが, ここでは一般化して説明するため  
標準化$Z=(\overline{X}-130/\sqrt{\frac{9}{14}})$を用いる. このことで上側100α%点を$z_α$で表すことができ, 臨界値を  
$$
P((\overline{X}-130)/\sqrt{\frac{9}{14}} \leq x)=0.05
$$

これを満たすx, つまり$x=z_{0.95}$と求めることができる.  
したがって

- $(\overline{X}-130)/\sqrt{\frac{9}{14}} \lt z_{0.95}$であれば帰無仮説を棄却
- $(\overline{X}-130)/\sqrt{\frac{9}{14}} \geq z_{0.95}$であれば帰無仮説を採択

これらを計算してみる. まずは検定統計量から

In [7]:
z=(s_mean - 130) / np.sqrt(9/14)
z #検定統計量

-1.932298779026813

In [11]:
rv = stats.norm()
z_095=rv.isf(0.95) #臨界値

In [13]:
print(z<z_095) #帰無仮説を棄却
print(z>=z_095) #採択

True
False


統計検定量を標本平均とした時と結果は同じ, p値を使ったものも試してみる.   

In [16]:
p_value = rv.cdf(z) #p値は累積分布関数を求めることができる.
print(p_value < 0.05) #有意水準より小さいか？小さければ棄却

True


### 11.1.2:片側検定と両側検定
前の例では「母平均は130gより少ない」という検定を行ったが  
「母平均は130gではない」という対立仮説で行うことも可能である.  
つまり「母平均は130gより多い」という仮説検定も行うことになる.  
この様な検定を**両側検定**といい, 片方の検定だけ行うものを**片側検定**という.  

両側検定では有意水準は両側α/2となる, 有意水準5%の両側検定を行う.  
検定統計量は片側検定の時と変わらない.  

In [17]:
z

-1.932298779026813

In [21]:
# 両側検定なので臨界値は標準正規分布の95%区間によって求めることができる
rv=stats.norm()
rv.interval(0.95)[0]<z, rv.interval(0.95)[1]>z

(True, True)

採択域に入っている, 片側検定の方が棄却しやすいということは押さえておく事.  
次はp値について両側検定のp値は, 上側と下側の両方の面積を考える必要があるため, 累積密度関数の値を二倍する.

In [24]:
rv.cdf(z)*2 > 0.05 #棄却されない

True

### 11.1.3:仮説検定における2つの過誤
仮説検定は標本を用いて確率的に結論を導いているため, 判断を謝ることがわかる.  
以下の二つの過誤がありうる.  

- 第一種の過誤：帰無仮説が正しい時に, 帰無仮説を棄却してしまう過誤 → 誤検出
- 第二種の過誤：対立仮説が正しい時に, 帰無仮説を採択してしまう過誤 → 検出漏れ

最初の例でみてみる.  
#### 第一種の過誤
「母平均が130gである」にも関わらず「母平均が130gより少ない」という結論を出してしまうという事.  
この様な例が起きる割合で発生してしまうかみてみる.実際に「平均は130g」の状況を考えているので,  
母集団の確率分布はN(130,9)である. 

In [25]:
rv=stats.norm(130, 3)

この母集団から14個の標本を取り出し仮説検定を行うという作業を10000回行い
, 第一種の過誤を起こす割合を導出してみる.  

In [27]:
c=stats.norm().isf(.95)
n_samples=10000
cnt=0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14),2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_-130)/np.sqrt(9/14)
    if z < c :
        cnt += 1
cnt/n_samples

0.052

大体5%の割合で誤検出してしまう様である.  
誤検出する確率を危険率と呼ばれ, 記号にはαが使われる.  
危険率は有意水準と一致する, つまりこの危険率を1%にしたければ有意水準を1%にする. 

#### 第二種の過誤
今回は「母平均が130gより少ない」にも関わらず「母平均は130gよりも少ない」という結論を出せないという事.  
この確率が幾つであるかを確認する. ここで母平均がが128gであるとする, つまり母集団の確率分布はN(128, 9)である.

In [41]:
rv=stats.norm(128,3)
c=stats.norm().isf(0.95)
n_samples=10000
cnt=0
for _ in range(n_samples):
    sample_=np.round(rv.rvs(14),2)
    s_mean_=np.mean(sample_)
    z=(s_mean_-130)/np.sqrt(9/14)
    if z>=c:
        cnt+=1

In [42]:
print(cnt/n_samples, 1.0-cnt/n_samples)

0.1954 0.8046


大体20%くらいの検出漏れが出てくる, ここでいう80%は**検出力**これは母平均が120gの時の方が検出力は高くなる.  
この検出漏れにはβが使われこのβは母平均に依存するので分析者が制御できない要素である.  

すなわち第一種は制御できて第二種は制御できないという非対称性がある. 

## 11.2:基本的な仮説検定
この節では正規分布の母平均や母分散に関する基本的な仮説検定をみていく.  

### 11.2.1:正規分布の母平均の検定(母分散既知)
この検定では母平均がある値$\mu_0$ではないと主張する検定である.  
今回は母分散が分かっているので最も単純な状況設定である. 最初の例で言うと母平均が130g出ないと言う主張を検証する.  

今回の仮説検定は次の様になる.

In [47]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)

    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')

    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p値は{p:.3f}')

In [48]:
pmean_test(sample,130,9)

帰無仮説を採択
p値は0.053


両側検定の例で計算した結果と同じになる.

### 11.2.2:正規分布の母分散の検定
母分散の検定は母分散がある値$\sigma_0^2$ではない事のを主張するための検定である.  
統計検定量に$Y=(n-1)s^2/\sigma_0^2$を用いて, カイ二乗分布に従う事を利用する.

In [49]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')

    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p値は{p:.3f}')

In [50]:
#最初の例での標本データを実行してみる, ここでは分散を9とする
pvar_test(sample,9)

帰無仮説を採択
p値は0.085


### 11.2.3:正規分布の母平均の検定(母分散未知)
この場合の母平均の検定は**1標本のt検定**とも呼ばれ,  
**t検定統計量**と呼ばれる
$$
t=(\overline{X}-\mu_0)/\sqrt{\frac{s^2}{n}}
$$
を用いる. 

In [51]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')

    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p値は{p:.3f}')

In [55]:
pmean_test(sample,130)
#scipyでもt検定は可能
t,p=stats.ttest_1samp(sample,130)
t,p

帰無仮説を採択
p値は0.169


(-1.4551960206404198, 0.16933464230414275)

## 11.3:2標本問題に関する仮説検定
ここまでは1つの母集団に関する検定を考えてきた.  
ここから二つの母集団に関する問題, **2標本問題**を考えていく.  
2標本間の様々な関係性を調べることができる.  

2標本の代表値の間に差があるかの検定を扱う.  
代表値の差の検定は母集団に正規分布の仮定できるか, データに対応があるかで4つに分類できる.  
それぞれで検定の仕方が異なる.(教科書参照)

ここでデータに対応があるとは2つのデータが同一の個体を異なる条件で測定したデータになっていることを言う.  
e.g.
対応あり：薬剤投与した前後の同個体の値を標本として用いる場合
対応なし：A組とB組の生徒の点数を標本として用いる場合

### 11.3.1:対応のあるt検定
筋トレ前後で集中力に差があるかを20人に試す.

In [56]:
training_rel = pd.read_csv('../data/ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head()

(20, 2)


,前,後
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


前後の平均点を比較すれば良さそう

- 帰無仮説：$\mu_{after}-\mu_{bfore} = 0$
- 対立仮説：$\mu_{after}-\mu_{bfore} \neq 0$

今回はデータに対応があるので個々のデータで差を考えることができる.

In [57]:
training_rel['差'] = training_rel['後'] - training_rel['前']
training_rel.head()

,前,後,差
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


もし筋トレに集中力に影響させる要因がないならば差の平均は0になるはずこれを差の平均を$\mu_{diff}$として

- 帰無仮説：$\mu_{diff}=0$
- 対立仮説：$\mu_{diff}\neq0$

と言い換えることができる.  
さらに差を計算している元の分布が正規分布にしたがっているので差も正規分布にしたがっていると仮定できるので,  
1標本のt検定に帰着できる.

In [58]:
t, p = stats.ttest_1samp(training_rel['差'], 0)
p

0.04004419061842953

有意水準を下回ったので帰無仮説は棄却される.  
scipyでは前後の値を用いてそのまま検定できる関数もあるのでそれの
方が便利かも

In [59]:
t, p = stats.ttest_rel(training_rel['後'], training_rel['前'])
p

0.04004419061842953

### 11.3.2:対応のないt検定
2標本に正規分布を仮定できるがデータに対応がないもの.  
A組とB組の筋トレ前後の集中力について

In [60]:
training_ind = pd.read_csv('../data/ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


今回は対応のないデータなので差をとっても意味がない
今回は検定統計量には以下のtを用いる.  

$$
t=\frac{((\overline{X}-\overline{Y})-(\mu_1-\mu_2))}{\sqrt{\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}}}
$$

このtは自由度が

$$
\frac{\left( \frac{s_1^2}{n_1} + \frac{s_2^2}{n_2} \right)^2}
{\frac{ \left( \frac{s_1^2}{n_1} \right)^2}{n_1 - 1} + 
\frac{\left( \frac{s_2^2}{n_2}\right)^2}{n_2 - 1}}
$$

のt分布に従う. これをウェルチの方法という.  
stats.ttest_indで計算できる.

In [64]:
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'],
                       equal_var=False) #Falseとすることでウェルチの方法となる.
p<0.05 #Trueなら棄却, Flaseなら採択

False

有意な差あるとは言えないっぽい

### ウィルスコクソンの符号付き順位検定
データに対応があり差に正規分布を仮定できない場合の**中央値の差**の検定である.  
先ほどのデータの6行を用いていく.  

In [65]:
training_rel = pd.read_csv('../data/ch11_training_rel.csv')
toy_df = training_rel[:6].copy()
toy_df

,前,後
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


対応のあるデータなので差に注目していく.

In [66]:
diff = toy_df['後'] - toy_df['前']
toy_df['差'] = diff
toy_df

,前,後,差
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


ここからが方法が異なる, 順位を用いて検定を行う.  
最初に差の絶対値が小さいものから順に順位をつけていく. 

In [67]:
rank = stats.rankdata(abs(diff)).astype(int) #rankkdata関数が便利
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [69]:
r_minus = np.sum((diff < 0) * rank) # 差がminusの順位の和
r_plus = np.sum((diff > 0) * rank) # 差がplusの順位の和

r_minus, r_plus #これらの小さい方が統計検定量になる今回は8

(8, 13)

この検定量が臨界値より小さい場合に帰無仮説が棄却される様な片側検定を行う.  
ここでなぜこの方法で検定できるのかを極端な例で説明する.  

In [70]:
toy_df['後'] = toy_df['前'] + np.arange(1, 7)
diff = toy_df['後'] - toy_df['前']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['差'] = diff
toy_df['順位'] = rank
toy_df #全員が集中力が向上している結果になっている

,前,後,差,順位
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [71]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus #差に偏りがあるとどちらかは値が小さくなる

(0, 21)

次に上がった人も下がった人もいるという状況を考える.

In [72]:
toy_df['後'] = toy_df['前'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['後'] - toy_df['前']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['差'] = diff
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [74]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus # どっちもどっちな値になる, 検定統計量は大きめの11になる

(11, 10)

In [76]:
T, p = stats.wilcoxon(training_rel['前'], training_rel['後']) #差をそのまま渡せる
p < 0.05 #棄却される有意である, 対応のあるt検定の時と同じ結論

True

ウィルコクソンの符号付き順位検定は母集団が正規分布に従う場合でも使う事ができる.  
しかし対応ありt検定の方が検定力が高い.(教科書参照)  

### マン・ホイットニーのU検定
データに対応がなく正規分布を仮定できない場合の中央値の差の検定.  
先ほどのデータの6行を用いていく. 

In [77]:
training_ind = pd.read_csv('../data/ch11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [79]:
rank = stats.rankdata(np.concatenate([toy_df['A'],
                                      toy_df['B']]))
rank_df = pd.DataFrame({'A': rank[:5],
                        'B': rank[5:10]}).astype(int)
rank_df #この順位和を用いて検定統計量を出していく

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


In [81]:
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2 #ここでこの式は検定統計量がAにベストな順位が集まった時に0になるもの
u #発想的にはAにいい順位が集まっているほど検定統計量は小さくなるという考え方

7.0

In [82]:
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'],
                          alternative='two-sided') #U検定はこの関数で行う事ができる
p < 0.05 #採択される

False

### 11.3.5:カイ二乗検定
カイ二乗分布が用いられる独立性を検定する.
e.g.
広告と購入どうしがそれぞれ独立か影響し合うかなど

In [83]:
ad_df = pd.read_csv('../data/ch11_ad.csv')
n = len(ad_df)
print(n)
ad_df.head()

1000


,広告,購入
0,B,しなかった
1,B,しなかった
2,A,した
3,A,した
4,B,しなかった


In [89]:
ad_cross = pd.crosstab(ad_df['広告'], ad_df['購入'])
ad_cross #分割表, クロス集計表(観測度数)

購入,した,しなかった
広告,,
A,49,351
B,51,549


In [86]:
ad_cross['した'] / (ad_cross['した'] + ad_cross['しなかった']) #Aの方が良さそうだが…

広告
A    0.1225
B    0.0850
dtype: float64

In [87]:
n_yes, n_not = ad_cross.sum()
n_yes, n_not #購入した人, してない人

(100, 900)

In [88]:
n_adA, n_adB = ad_cross.sum(axis=1)
n_adA, n_adB #A見た, B見た

(400, 600)

これらの合計を用いていく, ABが独立ならば分割表はどうなるべきかを見ていくこの様な時の度数を  
**期待度数**といい実際の度数を**観測度数という**

In [90]:
ad_ef = pd.DataFrame({'した': [n_adA * n_yes / n,
                              n_adB * n_yes / n],
                      'しなかった': [n_adA * n_not / n,
                                   n_adB * n_not / n]},
                      index=['A', 'B'])
ad_ef #期待値度数

,した,しなかった
A,40.0,360.0
B,60.0,540.0


In [91]:
y = ((ad_cross - ad_ef) ** 2 / ad_ef).sum().sum() #カイ二乗検定ではこれら度数の乖離を測る事で検定を行う
y #近似的に自由度1のカイ二乗分布に従う

3.75

In [94]:
rv = stats.chi2(1) #近似的に自由度1のカイ二乗分布に従う
1 - rv.cdf(y) < 0.05 #採択→有意な差は認められない

False

In [95]:
chi2, p, dof, ef = stats.chi2_contingency(ad_cross,
                                          correction=False) #scipyでもできるよ
chi2, p, dof

(3.75, 0.052807511416113395, 1)

In [96]:
ef

array([[ 40., 360.],
       [ 60., 540.]])